In [14]:
import pandas as pd
import re
dataframe = pd.DataFrame(["1; 2 23",
"11213; Something Sothing Elisse",
"144141; Something_2 Something Else",
"40; Something_2 Somethink Else",
"80; Something_2 Somepenguin Else"], columns=['row'])

In [15]:
dataframe

,row
0,1; 2 23
1,11213; Something Sothing Elisse
2,144141; Something_2 Something Else
3,40; Something_2 Somethink Else
4,80; Something_2 Somepenguin Else


In [16]:
data = pd.DataFrame(columns=['number', 'string'])
for row in dataframe.row:
    number, string = row.split('; ')
    data = data.append({'number': int(number),'string': string}, ignore_index=True)
    

In [17]:
data

,number,string
0,1,2 23
1,11213,Something Sothing Elisse
2,144141,Something_2 Something Else
3,40,Something_2 Somethink Else
4,80,Something_2 Somepenguin Else


In [18]:
data.sort_values('number')

,number,string
0,1,2 23
3,40,Something_2 Somethink Else
4,80,Something_2 Somepenguin Else
1,11213,Something Sothing Elisse
2,144141,Something_2 Something Else


In [19]:
data.sort_values('string', ascending=False)

,number,string
3,40,Something_2 Somethink Else
2,144141,Something_2 Something Else
4,80,Something_2 Somepenguin Else
1,11213,Something Sothing Elisse
0,1,2 23


In [20]:
def distance(a, b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n, m)) space
        a, b = b, a
        n, m = m, n

    current_row = range(n + 1)  # Keep current and previous row, not entire matrix
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [21]:
data['lev_anything_else'] = data.apply(lambda x: distance(x['string'], 'Anything else'), axis=1)

In [22]:
data

,number,string,lev_anything_else
0,1,2 23,12
1,11213,Something Sothing Elisse,14
2,144141,Something_2 Something Else,16
3,40,Something_2 Somethink Else,16
4,80,Something_2 Somepenguin Else,18


In [23]:
data.sort_values('lev_anything_else')

,number,string,lev_anything_else
0,1,2 23,12
1,11213,Something Sothing Elisse,14
2,144141,Something_2 Something Else,16
3,40,Something_2 Somethink Else,16
4,80,Something_2 Somepenguin Else,18


In [31]:
def find_replacement(m):
    return replacement[m.group(1)]

In [32]:
replacement = {'Something': 'Who knows'}

regex = r'({})'.format(r'|'.join(re.escape(w) for w in replacement))

for i, row in data.iterrows():
    if distance(row['string'], 'Anything else') > 2:
        data['string'][i] = re.sub(regex, find_replacement, row['string'])

C:\Users\Miron\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [33]:
data

,number,string,lev_anything_else
0,1,2 23,12
1,11213,Who knows Sothing Elisse,14
2,144141,Who knows_2 Who knows Else,16
3,40,Who knows_2 Somethink Else,16
4,80,Who knows_2 Somepenguin Else,18
